The Python packages are firstly imported

In [36]:
import numpy as np
import pandas as pd

The files to use are imported and the formatting is adjusted by excluding irrelevant rows.

In [38]:
df2017 = pd.read_excel('20200211 EU_2007_2013_Payment_COHESION FINAL (2).xls',
                      sheet_name='Cumulative Net Pay and Closure ',
                      header=3)

df2016 = pd.read_excel('20200211 EU_2007_2013_Payment_COHESION FINAL (2).xls',
                      sheet_name='Cumulative Net Pay and Closure ',
                      header=3)

files = ['./OD_07_13_EU_payments_31122018.xlsx',
        './OD_07_13_EU_payments 31-12-2019.xlsx',
        './OD_07_13_EU_payments 31-12-2020.xlsx',
        './OD_07_13_EU_payments 31-12-2021.xlsx',
        './OD_07_13_EU_payments 14 -07- 2023.csv']

The file related to the agricultural funds is also imported. Nomenclature is adjusted to ensure consistency with the other files.

In [39]:
df_EAFRD = pd.read_excel('./20200123 EAFRD get_all_payments.xlsx', 
                         sheet_name='CCI by year',
                        header=2)

df_EAFRD = df_EAFRD.rename(columns={'Budget \nYear':'Year', 'Sum of Paid ':'Net payments'})
df_EAFRD['Fund'] = 'EAFRD'
df_EAFRD = df_EAFRD.loc[(df_EAFRD.Year>2015)&(df_EAFRD.Year<2020)].reset_index(drop=True)

Finally, the most updated version of the data file used for the payments regionalisation in stata is also imported.

In [40]:
previous_df = pd.read_excel('./Stata2Python/Datafiles/Base_data/170801 Old_data_detailed_CLEAN.xlsx')

The column names in the data files are harmonised. The year they refer to is added as an extra column.

In [44]:
df2017['2017'] = df2017['Cumulative Net Payment in 2017']

df2017_reduced = df2017[['CCI','Category','Fund','2017']]

df2017_reduced = df2017_reduced.rename(columns={'Category':'Objective',
                                               '2017':'Total net payments'})

df2017_reduced['Year'] = 2017

df2016['2016'] = df2016['Cumulative Net Payment in 2016']

df2016_reduced = df2016[['CCI','Category','Fund','2016']]

df2016_reduced = df2016_reduced.rename(columns={'Category':'Objective',
                                               '2016':'Total net payments'})

df2016_reduced['Year'] = 2016

df2017_reduced = pd.concat([df2016_reduced, df2017_reduced])

df_list = [df2017_reduced]

Years = list(range(2018,2023))

for ifi,f in enumerate(files):
    print(f)
    print(Years[ifi])
    try:
        df = pd.read_excel(f,
                           sheet_name='details')
    except ValueError:
        df = pd.read_csv(f)
        
    df['Year'] = Years[ifi]
    
    try:
        df['Total net payments'] = df['Total net payments'].str.replace(',', '').astype(float)
    except AttributeError:
        pass
    
    # print(len(df))
        
    df_list.append(df)

# df_list.append(df_EAFRD)
               
df_1722 = pd.concat(df_list)

df_1722 = df_1722[['Year','CCI','Title','Fund','Objective','Total net payments']]

Net yearly figures for the payments are obtained by subtraction from the yearly cumulative figures. Negative payments are capped to zero.

In [47]:
df_1722 = df_1722.set_index(['CCI','Objective','Fund']).sort_values(by='Year').sort_index()

df_1722['Net payments'] = df_1722.groupby(df_1722.index)['Total net payments'].diff().fillna(df_1722['Total net payments'])

df_1722.loc[(df_1722['Net payments']<0),'Net payments'] = 0

df_1722 = df_1722.loc[df_1722.Year>2016].drop('Total net payments',axis=1).reset_index()

The agricultural fund is eventually harmonised to the standard format and concatenated to the joined data set.

In [51]:
df_1722_noagri = df_1722.drop(df_1722[(df_1722.Fund=='EAFRD')&(df_1722.Year<2020)].index)

df_EAFRD = df_EAFRD.drop(columns='MS')

df_1722_agr = pd.concat([df_1722_noagri,df_EAFRD])

df_1722_agr = df_1722_agr.rename(columns={'Title':'CCITitle'})
df_1722_agr['ProgrammingPeriod']='2007-2013'
df_1722_agr['StageOfexpenditure']='Paid'

The data set is eventually pivoted and some funding schemes are renamed for consistency (EFF, and EAFRD).

In [52]:
df_1722_pivoted = pd.pivot_table(df_1722_agr, 
               values='Net payments', 
               index=['Year', 'CCI','ProgrammingPeriod','StageOfexpenditure'],
                columns=['Fund'], 
               aggfunc="sum",
              fill_value=0).reset_index()

In [53]:
df_1722_pivoted = df_1722_pivoted.rename(columns={'EFF':'FIFG', 'EAFRD':'EAGGF'})

The data set is concatenated to the updated figures for 2007-2013 coming from the previous contract.

In [57]:
previous_df_0713 = previous_df.loc[previous_df.ProgrammingPeriod=='2007-2013']

mapping_dict = previous_df_0713.set_index('CCI')['NUTS2Code'].to_dict()

df_1722_pivoted['NUTS2Code'] = df_1722_pivoted['CCI'].map(mapping_dict)

df_final = pd.concat([previous_df,
                     df_1722_pivoted])

Missing data are replaced with zeroes and the final columns are selected.

In [ ]:
df_final.loc[df_final['EAGGF'].apply(lambda x: isinstance(x, str)),'EAGGF'] = 0
df_final[['CF','ESF','ERDF','FIFG','EAGGF']] = df_final[['CF','ESF','ERDF','FIFG','EAGGF']].fillna(0)

df_final = df_final[[              'Year',  'ProgrammingPeriod',                'CCI',
                 'CCITitle',          'Objective',          'NUTS1Code',
         'NUTS1Description',          'NUTS2Code',   'NUTS2Description',
       'StageOfexpenditure',              'AllSF',                 'CF',
                    'EAGGF',               'ENPI',               'ERDF',
                      'ESF',               'FIFG',                'IPA',
                     'ISPA',            'Country']]

df_final = df_final.loc[~df_final.CCI.astype(str).str.isdigit()]
df_final[['CCITitle','NUTS1Code','NUTS1Description','NUTS2Code','NUTS2Description','Country']] = \
                    df_final[['CCITitle',
                              'NUTS1Code',
                              'NUTS1Description',
                             'NUTS2Code',
                             'NUTS2Description',
                             'Country']].astype(str)
df_final['CCITitle'] = df_final['CCITitle'].fillna('Data not Available')

df_final.loc[df_final.Country.isin(['0','nan']),'Country']=df_final.loc[df_final.Country.isin(['0','nan'])]['CCI'].str[4:6]

Differences for the yearly updated data for the agricultural funds and the data available from the previous contract are attributed to the year 2015, for which figures were not completed.

In [63]:
agri_extra_2015 = df_1722.loc[(df_1722.Fund=='EAFRD')&(df_1722.Year==2019)].groupby(['CCI'])['Net payments'].sum()
agri_extra_diff = (agri_extra_2015-df_final.loc[df_final.ProgrammingPeriod=='2007-2013'].groupby('CCI')['EAGGF'].sum()).dropna()

agri_extra = agri_extra_diff/df_final.loc[(df_final.ProgrammingPeriod=='2007-2013')&(df_final.Year==2015)&(df_final.CCI.isin(agri_extra_2015.index))].groupby('CCI').size()

df_final.loc[(df_final.ProgrammingPeriod=='2007-2013')&(df_final.Year==2015),'EAGGF']+=df_final.loc[(df_final.ProgrammingPeriod=='2007-2013')&(df_final.Year==2015),'CCI'].map(agri_extra)

A column for the cumulative payments across funding schemes is introduced. The title of the CCI identifier of the programme is capped to 32 characters to avoid inconsistent treatment between stata and python.

In [81]:
df_final['AllSF'] = df_final[['CF','ESF','ERDF','FIFG','EAGGF','IPA','ISPA','ENPI']].sum(axis=1)

df_final[['AllSF','CF','ESF','ERDF','FIFG','EAGGF','IPA','ISPA','ENPI']] = df_final[['AllSF','CF','ESF','ERDF','FIFG','EAGGF','IPA','ISPA','ENPI']].fillna(0)

df_final['CCITitle'] = df_final['CCITitle'].str.slice(0, 32)
df_final['CCITitle']= df_final['CCITitle'].str.replace('[^a-zA-Z0-9 ]', '', regex=True)

The output data is eventually exported to be used in the regionalisation algorithm.

In [277]:
df_final.to_excel('./Output/240201_Old_data_detailed_CLEAN.xlsx', index = False)

df_final.to_stata('./Output/Old_data_detailed_CLEAN.dta', version = 118, write_index = False)

df_final2015 = df_final.loc[df_final.Year>=2015].drop('Objective',axis=1)

df_final2015.to_excel('./Output/Update_15_22.xlsx', index = False)

df_final2015.to_stata('./Output/Update_15_22.dta', version = 118)